# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from train_raintomorrow import clean_data
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from azureml.widgets import RunDetails

from azureml.train.automl import AutoMLConfig
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.core import Pipeline


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
This AutoML experiment will be performed in order to investigate of *rain tomorrow* dataset. This dataset has xx columns and ... rows....

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-rain'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


# Create a Cluster

In [ ]:

#cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "capstone"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster=ComputeTarget(workspace=ws,name=cluster_name)
    print('Cluster already exists!')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
    cpu_cluster = ComputeTarget(ws,cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
data_rain=TabularDatasetFactory.from_delimited_files(path)

if "training" not in os.listdir():
    os.mkdir("./training")

x, y = clean_data(data_rain)### YOUR DATA OBJECT HERE ###

# split into train and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=42)

#data_train=pd.concat([x_train,y_train],axis=1)
#data_test=pd.concat([x_test,y_test],axis=1)
data_train=x_train.copy()
data_test=x_test.copy()
data_train['y']=y_train
data_test['y']=y_test
ds=data_train.to_csv('training/train.csv')
ds=ws.get_default_datastore()
ds.upload(src_dir='training',target_path='training',overwrite=True)
ds_train=Dataset.Tabular.from_delimited_files(path=[(ds,'training/train.csv')])



In [ ]:
ds_train

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
# let us run the automl for 20 minutes
# 
automl_settings = {experiment_timeout_minutes=20,
    task='classification',
    primary_metric='accuracy',
    compute_target=cpu_cluster,
    training_data=ds_train,
    label_column_name='y',
    n_cross_validations=5,
    enable_early_stopping=True}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)


ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True


In [ ]:
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [ ]:
# TODO: Submit your experiment
pipeline_run = experiment.submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(pipeline_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)
print('best model:',best_model_output)


In [ ]:
#TODO: Save the best model
# I am not sure whether saving like that will work
saved_model=best_model_output.register_model(model_name='bestmodel_automl',model_path='')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
cpu_cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
